<a href="https://colab.research.google.com/github/Jojo18-20/AI_course_submission/blob/main/module7_assignment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Johanna Schaefer

In [37]:
import pandas as pd
#!pip install scikit-learn==1.3.2
from sympy.printing.tensorflow import tensorflow
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.model_selection import GridSearchCV
#!pip install scikeras
from scikeras.wrappers import KerasRegressor
from tensorflow.keras.optimizers import Adam
from keras.models import load_model
import numpy as np
from tensorflow.keras.layers import LSTM
import matplotlib.pyplot as plt



In [64]:
# Define the URL from which to fetch the CSV data.
url = 'https://raw.githubusercontent.com/zhenliangma/Applied-AI-in-Transportation/master/Exercise_7_Neural_networks/Exercise7BikeSharing.csv'

# Use pandas to read the CSV data from the specified URL and store it in a DataFrame 'df'.
df = pd.read_csv(url)
# Display the first 10 rows of the DataFrame 'df'.
#df = pd.read_csv('Exercise7BikeSharing.csv')
df.head(10)

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0000,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0000,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0000,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0000,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0000,0,1,1
5,6,2011-01-01,1,0,1,5,0,6,0,2,0.24,0.2576,0.75,0.0896,0,1,1
6,7,2011-01-01,1,0,1,6,0,6,0,1,0.22,0.2727,0.80,0.0000,2,0,2
7,8,2011-01-01,1,0,1,7,0,6,0,1,0.20,0.2576,0.86,0.0000,1,2,3
8,9,2011-01-01,1,0,1,8,0,6,0,1,0.24,0.2879,0.75,0.0000,1,7,8
9,10,2011-01-01,1,0,1,9,0,6,0,1,0.32,0.3485,0.76,0.0000,8,6,14


In [79]:
target = 'cnt'
df['hr_workingday'] = df['hr'] * df['workingday']
df['temp_windspeed'] = df['temp'] * df['windspeed']
features = [
    'temp','atemp','hum','windspeed','weathersit',
    'hr','weekday','workingday','holiday','season','yr', 'hr_workingday','temp_windspeed'
]

X = df[features].copy()
y = df[target].astype(float)


# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


# Scale features (important for NN)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)




In [76]:
# A super-fast baseline, just for context
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train_scaled, y_train)
y_pred_lr = lr.predict(X_test_scaled)

mse_lr = mean_squared_error(y_test, y_pred_lr) #,squared=False
mae_lr = mean_absolute_error(y_test, y_pred_lr)
r2_lr = r2_score(y_test, y_pred_lr)

print({"LR_MSE": mse_lr, "LR_MAE": mae_lr, "LR_R2": r2_lr})

{'LR_MSE': 19207.87013021526, 'LR_MAE': 104.18262034885642, 'LR_R2': 0.3934116337184278}


In [24]:
 def construct_network_model(layer_sizes,dropout_rate,input_dim,learning_rate):
  # Create a Sequential model, which is a linear stack of layers.
  model = Sequential()
  model.add(Input(shape=(input_dim,)))
  for size in layer_sizes:
        model.add(Dense(size, activation='relu'))
        if dropout_rate > 0:
            model.add(Dropout(dropout_rate))
  model.add(Dense(1))  # Regression
  optimizer = Adam(learning_rate=learning_rate)
  model.compile(optimizer=optimizer, loss='mae', metrics=['mae'])
  return model

In [85]:
model_A = KerasRegressor(model=construct_network_model, verbose=0,model__input_dim=X_train_scaled.shape[1])
early_stop = EarlyStopping(monitor='val_mae', patience=5, restore_best_weights=True)


param_grid_A = {
    "model__learning_rate": [0.0005, 0.001, 0.005],
    "batch_size": [32, 64],
    "epochs": [250],
    "model__layer_sizes": [[64, 32]], #will be varied later
    "model__dropout_rate": [0.0], #will be varied later
}


grid_A = GridSearchCV(model_A, param_grid_A, cv=2, n_jobs=-2,scoring='neg_mean_absolute_error', verbose=2)
grid_A.fit(X_train_scaled, y_train,callbacks=[early_stop],validation_split=0.2)

Fitting 2 folds for each of 6 candidates, totalling 12 fits
[CV] END batch_size=32, epochs=250, model__dropout_rate=0.0, model__layer_sizes=[64, 32], model__learning_rate=0.0005; total time= 2.7min
[CV] END batch_size=32, epochs=250, model__dropout_rate=0.0, model__layer_sizes=[64, 32], model__learning_rate=0.0005; total time= 2.6min
[CV] END batch_size=32, epochs=250, model__dropout_rate=0.0, model__layer_sizes=[64, 32], model__learning_rate=0.001; total time= 2.6min
[CV] END batch_size=32, epochs=250, model__dropout_rate=0.0, model__layer_sizes=[64, 32], model__learning_rate=0.001; total time= 2.6min
[CV] END batch_size=32, epochs=250, model__dropout_rate=0.0, model__layer_sizes=[64, 32], model__learning_rate=0.005; total time= 2.9min
[CV] END batch_size=32, epochs=250, model__dropout_rate=0.0, model__layer_sizes=[64, 32], model__learning_rate=0.005; total time= 2.7min
[CV] END batch_size=64, epochs=250, model__dropout_rate=0.0, model__layer_sizes=[64, 32], model__learning_rate=0.000

,estimator,KerasRegresso...input_dim=13 )
,param_grid,"{'batch_size': [32, 64], 'epochs': [250], 'model__dropout_rate': [0.0], 'model__layer_sizes': [[64, 32]], ...}"
,scoring,'neg_mean_absolute_error'
,n_jobs,-2
,refit,True
,cv,2
,verbose,2
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,model,<function con...x7dfd0d892660>


In [89]:
best_lr = grid_A.best_params_["model__learning_rate"]
best_batch = grid_A.best_params_["batch_size"]

model_B = KerasRegressor(
    model=construct_network_model,
    input_dim=X_train_scaled.shape[1],verbose=0
)
param_grid_B = {
    "model__learning_rate": [best_lr],
    "batch_size": [best_batch],
    "epochs": [200],
    "model__layer_sizes": [
        [128, 64, 32],
        [64, 32],
        [64, 64],
        [256, 128, 64]],
    "model__dropout_rate": [0.0, 0.2, 0.5],
}

grid_B = GridSearchCV(model_B, param_grid_B, cv=2, scoring='neg_mean_absolute_error', verbose=2)

grid_B.fit(X_train_scaled, y_train,callbacks=[early_stop],validation_split=0.2)
grid_B.best_estimator_.model_.save("gridsearch_best_v2.keras")

Fitting 2 folds for each of 12 candidates, totalling 24 fits
[CV] END batch_size=64, epochs=200, model__dropout_rate=0.0, model__layer_sizes=[128, 64, 32], model__learning_rate=0.0005; total time= 1.2min
[CV] END batch_size=64, epochs=200, model__dropout_rate=0.0, model__layer_sizes=[128, 64, 32], model__learning_rate=0.0005; total time= 1.2min
[CV] END batch_size=64, epochs=200, model__dropout_rate=0.0, model__layer_sizes=[64, 32], model__learning_rate=0.0005; total time= 1.1min
[CV] END batch_size=64, epochs=200, model__dropout_rate=0.0, model__layer_sizes=[64, 32], model__learning_rate=0.0005; total time= 1.5min
[CV] END batch_size=64, epochs=200, model__dropout_rate=0.0, model__layer_sizes=[64, 64], model__learning_rate=0.0005; total time= 1.2min
[CV] END batch_size=64, epochs=200, model__dropout_rate=0.0, model__layer_sizes=[64, 64], model__learning_rate=0.0005; total time= 1.2min
[CV] END batch_size=64, epochs=200, model__dropout_rate=0.0, model__layer_sizes=[256, 128, 64], model

In [91]:
results = pd.DataFrame(grid_B.cv_results_)
results.sort_values(by='mean_test_score', ascending=False)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_batch_size,param_epochs,param_model__dropout_rate,param_model__layer_sizes,param_model__learning_rate,params,split0_test_score,split1_test_score,mean_test_score,std_test_score,rank_test_score
5,77.975667,3.266151,0.327348,0.039371,64,200,0.2,"[64, 32]",0.0005,"{'batch_size': 64, 'epochs': 200, 'model__drop...",-65.277822,-151.209058,-108.243440,42.965618,1
4,95.714161,3.284920,0.535683,0.195213,64,200,0.2,"[128, 64, 32]",0.0005,"{'batch_size': 64, 'epochs': 200, 'model__drop...",-51.420846,-167.512881,-109.466863,58.046018,2
11,117.642961,1.706560,0.363552,0.007684,64,200,0.5,"[256, 128, 64]",0.0005,"{'batch_size': 64, 'epochs': 200, 'model__drop...",-53.011560,-166.430440,-109.721000,56.709440,3
7,119.511679,0.057777,0.339093,0.003464,64,200,0.2,"[256, 128, 64]",0.0005,"{'batch_size': 64, 'epochs': 200, 'model__drop...",-51.898292,-167.800025,-109.849159,57.950866,4
8,93.469362,2.654406,0.423037,0.055618,64,200,0.5,"[128, 64, 32]",0.0005,"{'batch_size': 64, 'epochs': 200, 'model__drop...",-58.280129,-162.054580,-110.167355,51.887226,5
10,82.309813,11.476899,0.302097,0.008655,64,200,0.5,"[64, 64]",0.0005,"{'batch_size': 64, 'epochs': 200, 'model__drop...",-71.688386,-149.129973,-110.409180,38.720794,6
9,69.586259,0.088601,0.321291,0.045042,64,200,0.5,"[64, 32]",0.0005,"{'batch_size': 64, 'epochs': 200, 'model__drop...",-82.279550,-150.931431,-116.605491,34.325941,7
6,73.910118,0.117705,0.330243,0.034711,64,200,0.2,"[64, 64]",0.0005,"{'batch_size': 64, 'epochs': 200, 'model__drop...",-58.023602,-176.123431,-117.073517,59.049915,8
1,76.883821,10.634993,0.341784,0.035195,64,200,0.0,"[64, 32]",0.0005,"{'batch_size': 64, 'epochs': 200, 'model__drop...",-61.215512,-177.773052,-119.494282,58.278770,9
3,112.951768,0.377474,0.540513,0.187083,64,200,0.0,"[256, 128, 64]",0.0005,"{'batch_size': 64, 'epochs': 200, 'model__drop...",-54.744419,-195.354320,-125.049369,70.304951,10


In [94]:
filepath="weights.best.keras"
model_nn = load_model(filepath)
y_pred_nn = model_nn.predict(X_test_scaled)
mae_nn = mean_absolute_error(y_test, y_pred_nn)
mse_nn = mean_squared_error(y_test, y_pred_nn)
r2_nn = r2_score(y_test, y_pred_nn)
print(model_nn.summary())
print(f"Mean Absolute Error: {mae_nn}")
print(f"Mean Squared Error: {mse_nn}")
print(f"R-squared: {r2_nn}")

109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


Model: "sequential_551"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_1337 (Dense)              │ (None, 256)            │         3,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_639 (Dropout)           │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1338 (Dense)              │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_640 (Dropout)           │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1339 (Dense)              │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_641 (Dropout)           │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1340 (Dense)              │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 134,405 (525.02 KB)

 Trainable params: 44,801 (175.00 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 89,604 (350.02 KB)

None
Mean Absolute Error: 33.03076978108414
Mean Squared Error: 2898.969784121402
R-squared: 0.9403670476286414


first grid search

In [95]:
model = KerasRegressor(model=construct_network_model, verbose=0,model__input_dim=X_train_scaled.shape[1])
early_stop = EarlyStopping(monitor='val_mae', patience=5, restore_best_weights=True)


param_grid = {
    "model__layer_sizes": [
        [128, 64, 32],
        #[64, 32],
        #[64, 64],
        #[128, 128, 64],
        [256, 128, 64]
    ],
    "model__dropout_rate": [0.2],#[0.0, 0.2, 0.5],
    "batch_size": [32],#[32, 64],
    "model__learning_rate": [0.01],#[0.001, 0.01, 0.1],
    "epochs": [200]
}


grid = GridSearchCV(model, param_grid, cv=2, n_jobs=-2,scoring='neg_mean_absolute_error', verbose=2)
grid.fit(X_train_scaled, y_train,callbacks=[early_stop],validation_split=0.2)
grid.best_estimator_.model_.save("weights.best_v2.keras")


Fitting 2 folds for each of 2 candidates, totalling 4 fits
[CV] END batch_size=32, epochs=200, model__dropout_rate=0.2, model__layer_sizes=[128, 64, 32], model__learning_rate=0.01; total time= 2.3min
[CV] END batch_size=32, epochs=200, model__dropout_rate=0.2, model__layer_sizes=[128, 64, 32], model__learning_rate=0.01; total time= 2.2min
[CV] END batch_size=32, epochs=200, model__dropout_rate=0.2, model__layer_sizes=[256, 128, 64], model__learning_rate=0.01; total time= 2.6min
[CV] END batch_size=32, epochs=200, model__dropout_rate=0.2, model__layer_sizes=[256, 128, 64], model__learning_rate=0.01; total time= 2.7min


In [96]:
filepath="weights.best_v2.keras"
model_nn = load_model(filepath)
y_pred_nn = model_nn.predict(X_test_scaled)
mae_nn = mean_absolute_error(y_test, y_pred_nn)
mse_nn = mean_squared_error(y_test, y_pred_nn)
r2_nn = r2_score(y_test, y_pred_nn)
print(model_nn.summary())
print(f"Mean Absolute Error: {mae_nn}")
print(f"Mean Squared Error: {mse_nn}")
print(f"R-squared: {r2_nn}")

109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


Model: "sequential_614"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_1524 (Dense)              │ (None, 128)            │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_708 (Dropout)           │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1525 (Dense)              │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_709 (Dropout)           │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1526 (Dense)              │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_710 (Dropout)           │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1527 (Dense)              │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 36,485 (142.52 KB)

 Trainable params: 12,161 (47.50 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 24,324 (95.02 KB)

None
Mean Absolute Error: 97.94358392085525
Mean Squared Error: 18782.1040633107
R-squared: 0.6136447081387022


Train model with best parametesr again

LTMS

In [80]:
split_index = int(len(X) * 0.8)
X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]

scaler = StandardScaler()

X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

y_train = pd.Series(y_train.values, index=X_train.index)
y_test = pd.Series(y_test.values, index=X_test.index)

In [84]:
def create_sequences_new(X, y, window=6):
    X_seq, y_seq = [], []
    X = X.sort_index()
    y = y.sort_index()
    for i in range(len(X) - window):
        X_seq.append(X.iloc[i:i+window].values)
        y_seq.append(y.iloc[i+window])
    return np.array(X_seq), np.array(y_seq)


X_seq, y_seq = create_sequences_new(X_train_scaled, y_train, window=3)
X_test_seq, y_test_seq = create_sequences_new(X_test_scaled, y_test, window=3)

val_split_index = int(len(X_seq) * 0.8)
X_train_final, X_val = X_seq[:val_split_index], X_seq[val_split_index:]
y_train_final, y_val = y_seq[:val_split_index], y_seq[val_split_index:]

model_lstm = Sequential()
model_lstm.add(LSTM(64, input_shape=(X_seq.shape[1], X_seq.shape[2]), return_sequences=False))
model_lstm.add(Dropout(0.2))
model_lstm.add(Dense(1))  # Regression
model_lstm.compile(optimizer='adam', loss='mae', metrics=['mae'])

early_stop = EarlyStopping(monitor='val_mae', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_mae', factor=0.5, patience=3)

history_lstm = model_lstm.fit(
    X_train_final,
    y_train_final,
    validation_data=(X_val, y_val),
    epochs=200,
    batch_size=32,
    callbacks=[early_stop, reduce_lr],
    verbose=1)

y_pred_lstm = model_lstm.predict(X_test_seq)

model_lstm.save("lstm_model.keras")
mae_lstm = mean_absolute_error(y_test_seq, y_pred_lstm)
mse_lstm = mean_squared_error(y_test_seq, y_pred_lstm)
r2_lstm = r2_score(y_test_seq, y_pred_lstm)

print(f"MAE: {mae_lstm:.2f}, MSE: {mse_lstm:.2f}, R²: {r2_lstm:.2f}")

Epoch 1/200


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


348/348 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - loss: 146.8965 - mae: 146.8965 - val_loss: 243.2695 - val_mae: 243.2695 - learning_rate: 0.0010
Epoch 2/200
348/348 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 124.0758 - mae: 124.0758 - val_loss: 225.8122 - val_mae: 225.8122 - learning_rate: 0.0010
Epoch 3/200
348/348 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 112.2391 - mae: 112.2391 - val_loss: 212.4535 - val_mae: 212.4535 - learning_rate: 0.0010
Epoch 4/200
348/348 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 103.9197 - mae: 103.9197 - val_loss: 202.1635 - val_mae: 202.1635 - learning_rate: 0.0010
Epoch 5/200
348/348 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 98.4964 - mae: 98.4964 - val_loss: 194.2047 - val_mae: 194.2047 - learning_rate: 0.0010
Epoch 6/200
348/348 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 95.2591 - mae: 95.2591 - val_loss: 188.0500 - val_mae: 188.0500 - learning_rate: 0.0010
Epoch 7/200
348/348 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 92.5712 - mae: 92.5712 - val_loss: 183.3502 - val

In [ ]:
 # Create a scatter plot to visualize the relationship
  plt.figure(figsize=(8, 6))
  plt.scatter(y_test, y_pred_lstm, alpha=0.5)  # Plot actual vs. predicted values

  # Add labels and title
  plt.xlabel("Actual Values")
  plt.ylabel("Predicted Values")
  plt.title("Actual vs. Predicted Values")

In [83]:
results = pd.DataFrame([
    {'Model': 'Linear Regression', 'MSE': mse_lr, 'MAE': mae_lr, 'R2': r2_lr},
    {'Model': 'Feedforward NN', 'MSE': mse_nn, 'MAE': mae_nn, 'R2': r2_nn},
    {'Model': 'LSTM','MSE': mse_lstm, 'MAE': mae_lstm, 'R2': r2_lstm},
])
results

,Model,MSE,MAE,R2
0,Linear Regression,19207.870130,104.182620,0.393412
1,Feedforward NN,1954.468288,27.517488,0.938278
2,LSTM,13132.733735,69.739059,0.730085
